<a href="https://colab.research.google.com/github/MACHS92/03MIAR_10_A_2023-24_Algoritmos_de_Optimizacion/blob/main/Manuel_Chiriboga_Trabajo_Pr%C3%A1ctico_Algoritmos(V2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Manuel Alejandro Chiriboga Salazar  <br>
Url: https://github.com/MACHS92/03MIAR_10_A_2023-24_Algoritmos_de_Optimizacion/blob/main/Manuel_Chiriboga_Trabajo_Pr%C3%A1ctico_Algoritmos(V2).ipynb<br>
Google Colab: https://colab.research.google.com/drive/1uhCtXpNBSYqvIh4YpLesQ3waDD2hzYCs#scrollTo=hVbXYX-RfPWh <br>
Problema:

>3. Configuración de Tribunales

 Se precisa configurar tribunales de evaluación para un grupo de 15 alumnos que desean presentar su Trabajo Fin de Máster (TFM).

 Cada tribunal está compuesto por tres profesores, cada uno desempeñando uno de lossiguientes roles: Presidente, Secretario o Vocal.

Los profesores han indicado su disponibilidad horaria para participar en los tribunales de 15h a 21h durante la semana del 15 al 19 de abril:

Número de profesores : 10

Número de tribunales : 15

Disponibilidad/Roles : https://bit.ly/41QWk8o
- 1 indica que profesor tiene disponibilidad
- 0 en caso contrario










                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

Para resolver este problema de optimización debemos empezar por reconocer cual es la variable para optimizar. En el mismo en este caso sería el tiempo de cada profesor en los tribunales de TFM, evitando solapamientos , o incurriendo en choques de horarios o que algunos profesores tengan mas o menos horas de TFM.

##¿Como represento el espacio de soluciones?

Par modelar el espacio de solución tenemos  una matriz donde las columna representan los tribunales y las filas representan los profesores. Los valores de la matriz serían binarios, indicando si un profesor está asignado a un tribunal o no.



##¿Cual es la función objetivo?

 La función objetivo seria la minimización del solapamiento de horarios entre los tribunales. Por ejemplo, podríamos minimizar la cantidad total de horas en las que dos tribunales tienen al menos un profesor en común o también disminuir la repetición de profesores en un tribunal para que sea equitativa la distribución.  

##¿Como implemento las restricciones?

1.-Los profesores solo pueden ser asignados a tribunales durante sus horas de disponibilidad

2.-Cada tribunal debe tener exactamente tres profesores asignados.

3.-Cada profesor puede participar en un máximo de un tribunal a la vez.


In [ ]:
#Respuesta



#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

Como se observa en el programa anteriormente aplicado podemos notar que esta relaciona directamente  a los tres roles de los profesores y sus impedimentos horarios además impedimentos de los mismos roles . Por lo mismo Por lo tanto, la complejidad del algoritmo es exponencial en términos del número de tribunales, lo que lo hace computacionalmente costoso.

En cuanto al espacio de soluciones, cada tribunal puede tener diferentes combinaciones de profesores en cada uno de los tres roles. Dado que hay 10 profesores y 3 roles, el espacio de soluciones total para un tribunal es 103=1000. Dado que hay 15 tribunales en total, el espacio de soluciones completo es 100015, lo que representa una cantidad extremadamente grande de posibles configuraciones.

#Diseño
- ¿Que técnica utilizo? ¿Por qué?

En este caso después de analizar y probar un par de algoritmos concretamente genéticos con  la  librería deap de python  y backtracking con poda llegamos a la conclusión que la manera mas optima de conseguir una solución es backtracking con poda

In [ ]:
#Importaciones
import os
import csv
import math
import random
import pandas as pd
import numpy as np

In [ ]:
! git clone https://github.com/MACHS92/03MIAR_10_A_2023-24_Algoritmos_de_Optimizacion/blob/main/tribunal_horarios.csv
! git clone https://github.com/MACHS92/03MIAR_10_A_2023-24_Algoritmos_de_Optimizacion/blob/main/tribunal_roles.csv

Cloning into 'tribunal_horarios.csv'...
fatal: repository 'https://github.com/MACHS92/03MIAR_10_A_2023-24_Algoritmos_de_Optimizacion/blob/main/tribunal_horarios.csv/' not found
Cloning into 'tribunal_roles.csv'...
fatal: repository 'https://github.com/MACHS92/03MIAR_10_A_2023-24_Algoritmos_de_Optimizacion/blob/main/tribunal_roles.csv/' not found


In [ ]:
! ls

sample_data


In [ ]:
def generar_combinaciones(matriz_disponibilidad, matriz_roles):
    def es_valido(tribunal):
        for i in range(3):
            if sum(tribunal[:, i]) != 1:
                return False
        return True

    def poda(tribunal):
        for i in range(3):
            if sum(tribunal[:, i]) == 0:
                return False
        return True

    def backtrack(tribunal):
        nonlocal soluciones
        if tribunal.shape[0] == 0:
            if es_valido(tribunal):
                soluciones.append(tribunal)
            return
        if not poda(tribunal):
            return
        nuevo_tribunal = np.copy(tribunal)
        nuevo_tribunal[:, 0] = tribunal[:, 0] & matriz_roles[0]
        nuevo_tribunal[:, 1] = tribunal[:, 1] & matriz_roles[1]
        nuevo_tribunal[:, 2] = tribunal[:, 2] & matriz_roles[2]
        for i in range(nuevo_tribunal.shape[0]):
            if nuevo_tribunal[i].any():
                backtrack(np.delete(nuevo_tribunal, i, axis=0))

    soluciones = []
    backtrack(matriz_disponibilidad)
    return soluciones

def guardar_resultado(soluciones, nombres_profesores):
    data = {'Fecha': [], 'Hora': [], 'Presidente': [], 'Secretario': [], 'Vocal': []}
    for i, solucion in enumerate(soluciones):
        fecha = "15 de abril"
        hora = f"{15 + i}:00"
        for j, profesor in enumerate(solucion):
            if profesor[0] == 1:
                presidente = nombres_profesores[j]
            if profesor[1] == 1:
                secretario = nombres_profesores[j]
            if profesor[2] == 1:
                vocal = nombres_profesores[j]
        data['Fecha'].append(fecha)
        data['Hora'].append(hora)
        data['Presidente'].append(presidente)
        data['Secretario'].append(secretario)
        data['Vocal'].append(vocal)
    df = pd.DataFrame(data)
    df.to_csv('tribunales.csv', index=False)
disponibilidad = '''
0,1,0,1,1,1,0,1,1,1;
1,1,0,0,1,1,1,1,0,1;
1,1,1,1,0,1,1,1,1,0;
1,1,1,0,1,1,1,1,1,1;
0,0,0,1,0,1,1,1,0,1;
1,0,1,1,1,0,1,0,1,0;
1,0,1,0,1,0,1,0,0,1;
1,0,1,1,1,1,1,1,0,1;
0,1,1,0,1,1,1,1,1,0;
1,1,1,0,1,1,0,0,1,0;
1,1,0,1,0,1,1,1,1,0;
1,1,0,1,1,1,1,1,1,0;
1,0,1,1,1,1,0,1,1,0;
1,0,1,1,1,1,1,0,1,1;
1,1,1,0,1,1,0,1,1,1;
1,0,1,0,0,1,0,1,1,1;
0,0,1,1,1,0,1,1,0,0;
0,1,1,1,1,0,1,0,0,0;
1,1,1,1,0,1,0,0,0,1;
0,1,1,1,1,1,0,1,1,1;
1,0,1,1,1,1,1,1,1,1;
0,1,1,1,0,1,1,0,1,1;
1,1,0,0,1,1,0,1,1,0;
1,1,1,1,1,1,0,1,1,0;
1,1,1,1,1,0,0,1,1,1;
1,1,1,0,0,0,0,1,1,1;
1,1,0,1,1,1,1,1,1,1;
1,1,1,1,1,1,1,1,0,1;
1,1,0,1,1,1,1,0,1,1;
1,1,1,1,0,1,1,1,0,1;
1,1,1,1,1,1,1,1,1,1;
1,1,0,1,1,0,1,1,1,0;
1,1,1,1,1,1,1,0,1,0;
0,1,0,1,1,0,0,1,1,0;
0,1,1,0,0,1,1,0,1,1
'''




roles = '''
1,1,1;
1,1,1;
1,0,1;
0,1,1;
1,1,1;
1,1,1;
0,1,1;
0,1,1;
1,1,1;
1,1,1
'''

# Convertir las cadenas en matrices numpy

matriz_disponibilidad = np.matrix(disponibilidad.replace(",", " "))
matriz_roles = np.matrix(roles.replace(",", " "))
# Transponer matriz_disponibilidad
matriz_disponibilidad = matriz_disponibilidad.T
